In [ ]:
#engery scan runs
#scan	engery
#275	10011ev
#273	10017ev
#261	10020ev
#265	10026ev
#269	10032ev

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_scalebar.scalebar import ScaleBar
import pickle
from scipy.optimize import curve_fit
import scipy.ndimage

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
folder = '/content/drive/MyDrive/EuXFEL 3052/CODE/Energy Scan/'

Mounted at /content/drive


In [ ]:
# load single weak beam frame:
run = 275
weak_frame = 17
filename = f'{run}frames.pkl'
fname_full = folder + filename
with open(fname_full,'rb') as f:
  frames = pickle.load(f)
  test_img = frames[weak_frame,:,:]

In [ ]:
# load weak beam frames at each energy:
runlist = [275,273,265,261]
weak_beam_frames=[9,6,7,10]#same phi value
imgs = np.zeros([len(runlist),test_img.shape[0],test_img.shape[1]])
for idx, frame in enumerate(weak_beam_frame):
  fname = f'{runlist[idx]}frames.pkl'
  fname_full = folder + fname
  with open(fname_full,'rb') as f:
    img = pickle.load(f)
    imgs[idx,:,:] = img[frame,:,:]

In [ ]:
# plot weak beam frames to see amount we need to crop:
fig,ax = plt.subplots(1,len(runlist),figsize=(10,10))
for idx in range(len(runlist)):
  img = imgs[idx,:,:]
  ax[idx].imshow(img,vmin=np.percentile(img,10),vmax=np.percentile(img,97))
plt.tight_layout()

In [ ]:
# reordering by amount to crop:
runlist_cropped = [2,3,1,0]
rows_to_crop = [0,120,330,215]

# crop all frames
for idx in runlist_cropped:
  old_img = imgs[idx,:,:]
  cropped_img = old_img[rows_to_crop[idx]:,:]
  if idx == 2:
    max_length = cropped_img.shape[0]
    cropped_imgs = np.zeros([len(runlist),max_length,img.shape[1]])
    cropped_imgs[idx,:,:] = cropped_img
  else:
    cropped_imgs[idx,:,:] = cropped_img[:max_length,:]

# plot cropped images:
fig,ax = plt.subplots(1,len(runlist),figsize=(10,10))
for idx in range(len(runlist)):
  img = cropped_imgs[idx,:,:]
  im= ax[idx].imshow(img,vmin=np.percentile(img,10),vmax=np.percentile(img,97))
  ax[idx].set_title(f'Run {runlist[idx]}, frame {weak_beam_frame[idx]}')
plt.tight_layout()

In [ ]:
# check that summed intensities have gaussian fit as a function of energy?

energies = np.array([10011,10017,10026,10020])
summed_ints = np.zeros(len(runlist))
for idx in range(len(runlist)):
  summed_ints[idx] = sum(sum(cropped_imgs[idx,:,:]))

def gaussian(x, amplitude, mean, stddev):
    return amplitude * np.exp(-((x - mean) / stddev) ** 2 / 2)

p0 = [max(summed_ints),10016,np.std(energies)]
popt,pcov = curve_fit(gaussian,energies,summed_ints,p0=p0)
amp,mean,stdev = popt

x_fit = np.linspace(min(energies),max(energies),1000)

plt.figure()
plt.plot(energies,summed_ints,'o',color='black',label='Data')
plt.plot(x_fit,gaussian(x_fit,amp,mean,stdev),label='Gaussian Fit',color='red')

plt.xlabel('X-Ray Energy (eV)')
plt.ylabel('Summed Intensity (a.u.)')
plt.legend()
plt.title(r'Energy Scan for peaks chosen with guassian fit$')

plt.tight_layout()

In [ ]:
# sorting runs and frames based on energy:
sorted_indices = sorted(range(len(energies)),key=lambda i: energies[i],reverse=True)
energies_sorted = np.array([energies[i] for i in sorted_indices])
runlist_sorted = [runlist[i] for i in sorted_indices]
cropped_imgs_sorted = cropped_imgs[sorted_indices,:,:]

# converting from energies to d-spacing:
h = 6.6262e-34 # J s
c = 3e8 # m/s
theta = 35.04/2
E = energies_sorted*1.602e-19 # J
d = h*c/ (2*E*np.sin(np.deg2rad(theta)))*1e10 # angstroms

# accounting for scaling due to diffracted beam:
x_scale = 0.0369/(np.sin(np.deg2rad(35)))
y_scale = 0.0369
aspect = y_scale/x_scale


In [ ]:
# plot COM for a single run and energy COM
sigma = 3
idx = 3
fname = f'{runlist[idx]}frames.pkl'
fname_full = folder + fname
with open(fname_full,'rb') as f:
  frames = pickle.load(f)
fname_header = f'{runlist[idx]}framesheader.pkl'
fname_header_full = folder + fname_header
with open(fname_header_full,'rb') as f:
  frames_header = pickle.load(f)
varmid = 42.7152
varying_array = frames_header['SSRY'] - varmid
expanded_varying_array = varying_array[:,np.newaxis,np.newaxis]*np.ones(frames.shape)
filtered_frames = np.empty(frames.shape)
for i,frame in enumerate(frames):
  filtered_frames[i] = scipy.ndimage.gaussian_filter(frame,sigma)
filtered_frames[filtered_frames<0] = 10e-10
COM = np.average(expanded_varying_array, weights=filtered_frames, axis=0)
cmap_center = np.average(expanded_varying_array,weights=filtered_frames)
span_cmap = np.abs(varying_array[-1] - varying_array[0])/3

fig, ax = plt.subplots(1,2,figsize=(10,10))
im = ax[0].imshow(np.rot90(COM),vmin = cmap_center - span_cmap/2, vmax = cmap_center + span_cmap/2, cmap='PRGn',aspect=aspect)
cax = plt.colorbar(im,fraction=0.031,pad=0.04)
cax.set_label(r'$\Delta \Phi$ (degrees)')
ax[0].set_title('$\phi$ Center of Mass Scan')
ax[0].set_xticks([])
ax[0].set_yticks([])
scalebar = ScaleBar(0.00000003643, location='lower left', width_fraction=0.01)
ax[0].add_artist(scalebar)

sigma = 1
varying_array = d[1] - d
expanded_varying_array = varying_array[:,np.newaxis,np.newaxis]*np.ones(cropped_imgs_sorted.shape)
filtered_frames = np.empty(cropped_imgs_sorted.shape)
for i, frame in enumerate(cropped_imgs_sorted):
  filtered_frames[i] = scipy.ndimage.gaussian_filter(frame,sigma)
filtered_frames[filtered_frames<0] = 10e-10
COM = np.average(expanded_varying_array,weights=filtered_frames,axis=0)
cmap_center = np.average(expanded_varying_array,weights=filtered_frames)
span_cmap = np.abs(varying_array[-1] - varying_array[0])/3

im = ax[1].imshow(np.rot90(COM),vmin=cmap_center - span_cmap/2, vmax = cmap_center + span_cmap/2, cmap='RdBu', aspect=aspect)
cax = plt.colorbar(im,fraction=0.037,pad=0.04)
cax.set_label(r'$\Delta$ $\epsilon_{111}$ (Å)', rotation=90, labelpad=20)
cax.formatter.set_powerlimits((0, 0))
ax[1].set_title('Axial $\epsilon_{111}$ Strain Scan')
ax[1].set_xticks([])
ax[1].set_yticks([])
scalebar = ScaleBar(0.00000003643, location='lower left')
ax[1].add_artist(scalebar)

plt.tight_layout()

